In [2]:
import os
import io

from google.cloud import language_v1

In [3]:
os.environ["PROJECT_ID"] = "text-analysis-323506"

#### Create a new Google Cloud service account to access the APIs programmatically.

##### Following command creates a Google Cloud Service account, which is required to access APIs. This command should be run only once. When ran for the first time, it create a service account, details of which can be found in IAM & Admin section of GCP console

In [3]:
%%bash

# These should be run only once. These commands setup a cloud service account with required permissions to call APIs
gcloud iam service-accounts create my-api-sa --display-name "api account"

Created service account [my-api-sa].


#### Bind permission to access files stored in GCS

In [4]:
%%bash

gcloud iam service-accounts keys create ~/key.json --iam-account my-api-sa@${PROJECT_ID}.iam.gserviceaccount.com

created key [62359fcf802aaaebf5be875b91db872945b7c83b] of type [json] as [/home/jupyter/key.json] for [my-api-sa@text-analysis-323506.iam.gserviceaccount.com]


In [27]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.path.abspath("/home/jupyter/key.json")

#### Create a Google Cloud Natural Language api client object

In [28]:
language_client = language_v1.LanguageServiceClient()

### Important: 
#### Enable Cloud Natural Language API for this project in Google Cloud Platform console.

## Entity Analysis 

Analyzes input text for known entities. (proper names and common nouns)

In [29]:
text = "Google Cloud has wide range of APIs."

In [30]:
document = {'content': text, 'type_': language_v1.Document.Type.PLAIN_TEXT, 'language': 'en'}

# If file is stored in GCS, storage admin role must be bound to service account. Creating document:
# document = {"gcs_content_uri": 'gs://text-analysis-323506/document.txt', "type_": language_v1.Document.Type.PLAIN_TEXT, "language": 'en'}

In [31]:
response  = language_client.analyze_entities(request= {'document': document})

#### Dumping response to a text file

In [32]:
with open('./results/entity_analysis.txt', 'w') as fstream:
    fstream.write(str(response))

##### As it can be seen in result file, it even extracted wikipedia page for the phrase 'Google Cloud'

## Sentiment Analysis

#### Positive Example

In [33]:
# input text
text = "This movie is so good. Loved every second of it !!"

In [34]:
document = {'content': text, 'type_': language_v1.Document.Type.PLAIN_TEXT, 'language': 'en'}

# If file is stored in GCS, storage admin role must be bound to service account. Creating document:
# document = {"gcs_content_uri": 'gs://text-analysis-323506/document.txt', "type_": language_v1.Document.Type.PLAIN_TEXT, "language": 'en'}

In [35]:
response  = language_client.analyze_sentiment(request= {'document': document})

In [36]:
# Document Sentiment
response.document_sentiment

magnitude: 1.899999976158142
score: 0.8999999761581421

The score shows that the input text has a much positive sentiment. (Towards +1)

In [37]:
# Sentiments for each sentence in the input text
for sentence in response.sentences:
    print("Text: ")
    print(sentence.text.content)
    print("Sentiment: ")
    print(sentence.sentiment)

Text: 
This movie is so good.
Sentiment: 
magnitude: 0.8999999761581421
score: 0.8999999761581421

Text: 
Loved every second of it !!
Sentiment: 
magnitude: 0.8999999761581421
score: 0.8999999761581421



#### Dumping response to a text file

In [38]:
with open('./results/sentiment_analysis_positive.txt', 'w') as fstream:
    fstream.write(str(response))

#### Negative Example

In [39]:
# input text
text = "One of the worst movies ever!!. Please don't waste your time watching it.."

In [40]:
document = {'content': text, 'type_': language_v1.Document.Type.PLAIN_TEXT, 'language': 'en'}

# If file is stored in GCS, storage admin role must be bound to service account. Creating document:
# document = {"gcs_content_uri": 'gs://text-analysis-323506/document.txt', "type_": language_v1.Document.Type.PLAIN_TEXT, "language": 'en'}

In [41]:
response  = language_client.analyze_sentiment(request= {'document': document})

In [42]:
# Document Sentiment
response.document_sentiment

magnitude: 1.5
score: -0.699999988079071

The score shows that the input text has a much negative sentiment. (Towards -1)

In [43]:
# Sentiments for each sentence in the input text
for sentence in response.sentences:
    print("Text: ")
    print(sentence.text.content)
    print("Sentiment: ")
    print(sentence.sentiment)

Text: 
One of the worst movies ever!!.
Sentiment: 
magnitude: 0.800000011920929
score: -0.800000011920929

Text: 
Please don't waste your time watching it..
Sentiment: 
magnitude: 0.699999988079071
score: -0.699999988079071



#### Dumping response to a text file

In [44]:
with open('./results/sentiment_analysis_negative.txt', 'w') as fstream:
    fstream.write(str(response))

## Syntaxt Analysis

Analyzes the syntax of the text and provides sentence boundaries and tokenization along with part of speech tags, dependency trees, and other properties.

In [45]:
text = "Google Cloud has wide range of APIs."

In [46]:
document = {'content': text, 'type_': language_v1.Document.Type.PLAIN_TEXT, 'language': 'en'}

# If file is stored in GCS, storage admin role must be bound to service account. Creating document:
# document = {"gcs_content_uri": 'gs://text-analysis-323506/document.txt', "type_": language_v1.Document.Type.PLAIN_TEXT, "language": 'en'}

In [47]:
response  = language_client.analyze_syntax(request= {'document': document})

#### Dumping response to a text file

In [58]:
with open('./results/syntax_analysis.txt', 'w') as fstream:
    fstream.write(str(response))

## Text Classification

classifies text into different classes

Example 1:

In [49]:
text = "China has started shutting down schools, cancelling hundreds of flights and ramping up mass testings following a new outbreak of Covid-19"

In [50]:
document = {'content': text, 'type_': language_v1.Document.Type.PLAIN_TEXT, 'language': 'en'}

# If file is stored in GCS, storage admin role must be bound to service account. Creating document:
# document = {"gcs_content_uri": 'gs://text-analysis-323506/document.txt', "type_": language_v1.Document.Type.PLAIN_TEXT, "language": 'en'}

In [51]:
response = language_client.classify_text(request= {'document': document})

In [52]:
response

categories {
  name: "/Travel/Air Travel"
  confidence: 0.8999999761581421
}

Example 2:

In [53]:
text = "Mangoes are one of the tastiest fruits out there. I mean, who doesn't like mangoes?. There are a lot of different types of mangoes. Each \
        of them look and taste different."

In [54]:
document = {'content': text, 'type_': language_v1.Document.Type.PLAIN_TEXT, 'language': 'en'}

In [55]:
response = language_client.classify_text(request= {'document': document})

In [56]:
response

categories {
  name: "/Food & Drink/Food"
  confidence: 0.9800000190734863
}

#### Dumping response into a text file

In [57]:
with open('./results/text_classification.txt', 'w') as fstream:
    fstream.write(str(response))